## Vehicle Detection model 1

In [12]:
import os
import tensorflow as tf
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,InputLayer,Flatten,MaxPooling2D,Conv2D
from tensorflow.keras.activations import relu,sigmoid
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.metrics import binary_accuracy
from keras.losses import binary_crossentropy
from keras.optimizer_v1 import SGD
from keras.activations import softmax
from matplotlib import pyplot as plt


In [ ]:
train_src ="../input/vehicle-data-set/cardataset/train"
train_dst = "train"

test_src = "../input/vehicle-data-set/cardataset/test"
test_dst = "test"

os.mkdir(train_dst)
os.mkdir(test_dst)

for sub_dir in os.listdir(train_src):
    if not os.path.isdir(os.path.join(train_dst,sub_dir)):
        os.mkdir(os.path.join(train_dst,sub_dir))

    for img in os.listdir(os.path.join(train_dst,sub_dir)):
        image = load_img(img, target_size=(96,96))
        image.save(os.path.join(train_dst,sub_dir))

for sub_dir in os.listdir(test_src):
    if not os.path.isdir(os.path.join(test_dst,sub_dir)):
        os.mkdir(os.path.join(test_dst,sub_dir))

    for img in os.listdir(os.path.join(test_dst,sub_dir)):
        image = load_img(img, target_size=(96,96))
        image.save(os.path.join(test_dst,sub_dir))



In [ ]:
for class_no,emotion in emo_classes.items():
    new_train_src = os.path.join(train_src, class_no)
    new_train_dst = os.path.join(train_dst, emotion)

    for img in os.listdir(new_train_src):
        src = os.path.join(new_train_src, img)
        dst = os.path.join(new_train_dst, img)
        img_ = img_to_array(load_img(src, target_size=(64,64)))
        save_img(dst, img_)


    new_val_src = os.path.join(val_src, class_no)
    new_val_dst = os.path.join(val_dst, emotion)
    os.mkdir(f'val_class/{emotion}')

    for img in os.listdir(new_val_src):
        src = os.path.join(new_val_src, img)
        dst = os.path.join(new_val_dst, img)
        img_ = img_to_array(load_img(src, target_size=(64,64)))
        save_img(dst, img_)

In [ ]:
for img in os.listdir(car_dir):
    image_path = os.path.join(car_dir,img)
    image = img_to_array(load_img(path=image_path, target_size=(150,150)))


In [ ]:
genarator = ImageDataGenerator(rescale=1./255,
                               rotation_range=45,
                               horizontal_flip=True,
                               vertical_flip=True,
                               validation_split=0.2)

train_gen = genarator.flow_from_directory('../input',
                                          target_size=(96,96),
                                          batch_size=128,
                                          subset='training')

val_gen = genarator.flow_from_directory("../input",
                                        target_size=(96,96),
                                        batch_size=128,
                                        subset='validation')

In [9]:
model = Sequential()

model.add(InputLayer(input_shape=(96,96,3)))

model.add(Conv2D(filters=128, kernel_size=(3,3),activation=relu))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=196, kernel_size=(3,3),activation=relu))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),activation=relu))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),activation=relu))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),activation=relu))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),activation=relu))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),activation=relu))
model.add(MaxPooling2D(pool_size=(2,2)))

##classifier
model.add(Flatten())
model.add(Dense(units=1024, activation=relu))
model.add(Dense(units=512, activation=relu))
model.add(Dense(units=256, activation=relu))
model.add(Dense(units=128, activation=relu))
model.add(Dense(units=17, activation=sigmoid))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 94, 94, 128)       3584      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 47, 47, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 45, 45, 196)       225988    
                                                                 
 conv2d_26 (Conv2D)          (None, 43, 43, 128)       225920    
                                                                 
 conv2d_27 (Conv2D)          (None, 41, 41, 128)       147584    
                                                                 
 conv2d_28 (Conv2D)          (None, 39, 39, 128)       147584    
                                                      

In [11]:
plot_model(model,
           show_shapes=True,
           show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [ ]:


model.compile(optimizer=Adam,
              loss=softmax, #softmax
              metrics=['accuracy',binary_accuracy])

In [ ]:

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=np.sqrt(0.1))

check_point = ModelCheckpoint(filepath='alexNet.hdf5',
                              monitor='val_loss',
                              save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss',
                           patience=20,
                           restore_best_weights=True)

callbacks = [reduce_lr, check_point, early_stop]

In [ ]:
model.fit(train_gen,
          callbacks=callbacks,
          validation_data=val_gen,
          epochs=5)